In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageDraw, ImageFont
from scipy.misc import imread,imresize
from glob import glob
from PIL import Image, ImageDraw, ImageFont

In [2]:
#将图片二值化
def get_bin_table(threshold=230):
    table = []
    for i in range(256):
        if i < threshold:
            table.append(0)
        else:
            table.append(1)
    return table

In [3]:
#判断当前像素点是否为孤立点，即噪声点
def sun_9_region(img, x, y):
    #判断图片的长宽度下限
    sum = 0
    cur_pixel = img.getpixel((x, y))  #当前像素值
    width = img.width
    height = img.height
    
    if cur_pixel == 1:  #当前选择的像素点为白色，不统计领域的值
        return 0
    
    if y == 0:  #第一行
        if x == 0:  #当前为左上顶点，有4个领域
            sum = cur_pixel \
                    + img.getpixel((x, y+1)) \
                    + img.getpixel((x+1, y)) \
                    + img.getpixel((x+1, y+1))
            return 4-sum
        elif x ==  width-1:  #当前为右上顶点
            sum = cur_pixel \
                    + img.getpixel((x, y+1)) \
                    + img.getpixel((x-1, y)) \
                    + img.getpixel((x-1, y+1))
            return 4-sum
        else:  #最上非顶点，有6个领域
            sum = img.getpixel((x-1, y)) \
                + img.getpixel((x-1, y+1)) \
                + cur_pixel \
                + img.getpixel((x, y+1)) \
                + img.getpixel((x+1, y)) \
                + img.getpixel((x+1, y+1))
            return 6-sum
    elif y == height-1:  #最下面一行
        if x == 0:  #左下顶点
            sum == cur_pixel \
                    + img.getpixel((x+1, y)) \
                    + img.getpixel((x+1, y-1)) \
                    + img.getpixel((x, y-1))
            return 4-sum
        elif x == width-1:  #右下顶点
            sum = cur_pixel \
                    + img.getpixel((x, y-1)) \
                    + img.getpixel((x-1, y)) \
                    + img.getpixel((x-1, y-1))
            return 4 - sum
        else:  # 最下非顶点,6邻域
            sum = cur_pixel \
                    + img.getpixel((x-1, y)) \
                    + img.getpixel((x+1, y)) \
                    + img.getpixel((x, y-1)) \
                    + img.getpixel((x-1, y-1)) \
                    + img.getpixel((x+1, y-1))
            return 6 - sum
    else:  # 当前选择的像素点不在边界
        if x == 0:  # 左边非顶点
            sum = img.getpixel((x, y-1)) \
                    + cur_pixel \
                    + img.getpixel((x, y+1)) \
                    + img.getpixel((x+1, y-1)) \
                    + img.getpixel((x+1, y)) \
                    + img.getpixel((x+1, y+1))
            return 6 - sum
        elif x == width - 1:  # 右边非顶点
            # print('%s,%s' % (x, y))
            sum = img.getpixel((x, y-1)) \
                    + cur_pixel \
                    + img.getpixel((x, y+1)) \
                    + img.getpixel((x-1, y-1)) \
                    + img.getpixel((x-1, y)) \
                    + img.getpixel((x-1, y+1))
            return 6 - sum
        else:  # 具备9领域条件的
            sum = img.getpixel((x-1, y-1)) \
                    + img.getpixel((x-1, y)) \
                    + img.getpixel((x-1, y+1)) \
                    + img.getpixel((x, y-1)) \
                    + cur_pixel \
                    + img.getpixel((x, y+1)) \
                    + img.getpixel((x+1, y-1)) \
                    + img.getpixel((x+1, y)) \
                    + img.getpixel((x+1, y+1))
            return 9 - sum

In [4]:
#对当前图片中的每一个像素点进行判断,并收集
def collext_noise_point(img):
    noise_point_list = []
    for x in range(img.width):
        for y in range(img.height):
            res_9 = sun_9_region(img, x, y)
            if (0< res_9 <6) and img.getpixel((x, y)) == 0:
                noise_point_list.append((x, y))
    return noise_point_list

In [5]:
#根据噪声点的位置信息，消除二值化图片的噪声点
def remove_noise_pixel(img, noise_point_list):
    for item in noise_point_list:
        img.putpixel((item[0], item[1]), 1)

In [6]:
#把图片转化为99*80
def img_resize(img):
    img = img.resize((99,80))

In [7]:
#分割图像
def cut_imgs(img):
    child_img_list = []
    for i in range(3):
        x = i * 33
        y = 0
        child_img = img.crop((x, y, x+33, y+80))
        child_img_list.append(child_img)
    return child_img_list

In [49]:
'''
#单一图片示例
#读取图片,resize为99,80,并转为灰度图
image = Image.open('train/0.jpg')
image = image.resize((48, 38))
imgry = image.convert('L')

#将灰度图进行二值化
table = get_bin_table()
out = imgry.point(table, '1')

#进行5次降噪处理
for i in range(2):
    noise = collext_noise_point(out)
    remove_noise_pixel(out, noise)
out.show()

#把图像切割成3个单独的数字并存放在制定文件夹中
child_img_list = []
for i in range(3):
    x = i * 16
    y = 0
    child_img = out.crop((x, y, x+16, y+39))
    child_img_list.append(child_img)
    child_img.save('train_cut_img/1_'+str(i)+'.jpg','JPEG')
'''

''

In [12]:
#加载train_y
train_labels = pd.read_csv('train_labels.csv')
train_id = train_labels.pop('id')
train_y = train_labels.pop('y')

In [29]:
#加载并处理train
for i in range(10000):
    print('train:',i)
    img = Image.open('train/'+str(i)+'.jpg', 'r')
    img = img.resize((99, 80))
    #对图片进行二值化
    table = get_bin_table()
    img = img.convert('L')
    img = img.point(table, '1')
    #进行5次降噪处理 
    for j in range(5):
        noise = collext_noise_point(img)
        remove_noise_pixel(img, noise)
    #分割图片并存储
    width, height = img.size
    pixel_line = []  #图像列中的黑色像素点的个数
    for x in range(width):
        pix_y = 0
        for y in range(height):
            if img.getpixel((x, y)) == 0:  # 黑色像素点
                pix_y += 1
        pixel_line.append(pix_y)
    #找到黑色像素点最少的位置
    a = pixel_line[20]
    x1_pixel = 20
    b = pixel_line[55]
    x2_pixel = 55
    for m in range(20, 44):
        if a>=pixel_line[m]:
            a = pixel_line[m]
            x1_pixel = m
        else:
            continue
    for n in range(55,73):
        if b>=pixel_line[n]:
            b = pixel_line[n]
            x2_pixel = n
        else:
            continue
    #切割
    child_img1 = img.crop((0, 0, x1_pixel, 80))
    child_img1 = child_img1.resize((33,80))
    child_img2 = img.crop((x1_pixel, 0, x2_pixel, 80))
    child_img2 = child_img2.resize((33,80))
    child_img3 = img.crop((x2_pixel, 0, 99, 80))
    child_img3 = child_img3.resize((33,80)) 
    #将分割处理后的图片存入相应的文件夹中
    y0 = train_y[i]
    a = train_y[i]//100  #百位数
    child_img1.save('train_cut_img/'+str(a)+"/"+str(i)+'_0.jpg','JPEG')  #i:第几张图片, m:哪位数
    b = (train_y[i]//10)%10  #十位数
    child_img2.save('train_cut_img/'+str(b)+"/"+str(i)+'_1.jpg','JPEG')
    c = train_y[i]%10  #个位数
    child_img3.save('train_cut_img/'+str(c)+"/"+str(i)+'_2.jpg','JPEG')

train: 0
train: 1
train: 2
train: 3
train: 4
train: 5
train: 6
train: 7
train: 8
train: 9
train: 10
train: 11
train: 12
train: 13
train: 14
train: 15
train: 16
train: 17
train: 18
train: 19
train: 20
train: 21
train: 22
train: 23
train: 24
train: 25
train: 26
train: 27
train: 28
train: 29
train: 30
train: 31
train: 32
train: 33
train: 34
train: 35
train: 36
train: 37
train: 38
train: 39
train: 40
train: 41
train: 42
train: 43
train: 44
train: 45
train: 46
train: 47
train: 48
train: 49
train: 50
train: 51
train: 52
train: 53
train: 54
train: 55
train: 56
train: 57
train: 58
train: 59
train: 60
train: 61
train: 62
train: 63
train: 64
train: 65
train: 66
train: 67
train: 68
train: 69
train: 70
train: 71
train: 72
train: 73
train: 74
train: 75
train: 76
train: 77
train: 78
train: 79
train: 80
train: 81
train: 82
train: 83
train: 84
train: 85
train: 86
train: 87
train: 88
train: 89
train: 90
train: 91
train: 92
train: 93
train: 94
train: 95
train: 96
train: 97
train: 98
train: 99
train: 100

train: 755
train: 756
train: 757
train: 758
train: 759
train: 760
train: 761
train: 762
train: 763
train: 764
train: 765
train: 766
train: 767
train: 768
train: 769
train: 770
train: 771
train: 772
train: 773
train: 774
train: 775
train: 776
train: 777
train: 778
train: 779
train: 780
train: 781
train: 782
train: 783
train: 784
train: 785
train: 786
train: 787
train: 788
train: 789
train: 790
train: 791
train: 792
train: 793
train: 794
train: 795
train: 796
train: 797
train: 798
train: 799
train: 800
train: 801
train: 802
train: 803
train: 804
train: 805
train: 806
train: 807
train: 808
train: 809
train: 810
train: 811
train: 812
train: 813
train: 814
train: 815
train: 816
train: 817
train: 818
train: 819
train: 820
train: 821
train: 822
train: 823
train: 824
train: 825
train: 826
train: 827
train: 828
train: 829
train: 830
train: 831
train: 832
train: 833
train: 834
train: 835
train: 836
train: 837
train: 838
train: 839
train: 840
train: 841
train: 842
train: 843
train: 844
train: 845

train: 1460
train: 1461
train: 1462
train: 1463
train: 1464
train: 1465
train: 1466
train: 1467
train: 1468
train: 1469
train: 1470
train: 1471
train: 1472
train: 1473
train: 1474
train: 1475
train: 1476
train: 1477
train: 1478
train: 1479
train: 1480
train: 1481
train: 1482
train: 1483
train: 1484
train: 1485
train: 1486
train: 1487
train: 1488
train: 1489
train: 1490
train: 1491
train: 1492
train: 1493
train: 1494
train: 1495
train: 1496
train: 1497
train: 1498
train: 1499
train: 1500
train: 1501
train: 1502
train: 1503
train: 1504
train: 1505
train: 1506
train: 1507
train: 1508
train: 1509
train: 1510
train: 1511
train: 1512
train: 1513
train: 1514
train: 1515
train: 1516
train: 1517
train: 1518
train: 1519
train: 1520
train: 1521
train: 1522
train: 1523
train: 1524
train: 1525
train: 1526
train: 1527
train: 1528
train: 1529
train: 1530
train: 1531
train: 1532
train: 1533
train: 1534
train: 1535
train: 1536
train: 1537
train: 1538
train: 1539
train: 1540
train: 1541
train: 1542
trai

train: 2144
train: 2145
train: 2146
train: 2147
train: 2148
train: 2149
train: 2150
train: 2151
train: 2152
train: 2153
train: 2154
train: 2155
train: 2156
train: 2157
train: 2158
train: 2159
train: 2160
train: 2161
train: 2162
train: 2163
train: 2164
train: 2165
train: 2166
train: 2167
train: 2168
train: 2169
train: 2170
train: 2171
train: 2172
train: 2173
train: 2174
train: 2175
train: 2176
train: 2177
train: 2178
train: 2179
train: 2180
train: 2181
train: 2182
train: 2183
train: 2184
train: 2185
train: 2186
train: 2187
train: 2188
train: 2189
train: 2190
train: 2191
train: 2192
train: 2193
train: 2194
train: 2195
train: 2196
train: 2197
train: 2198
train: 2199
train: 2200
train: 2201
train: 2202
train: 2203
train: 2204
train: 2205
train: 2206
train: 2207
train: 2208
train: 2209
train: 2210
train: 2211
train: 2212
train: 2213
train: 2214
train: 2215
train: 2216
train: 2217
train: 2218
train: 2219
train: 2220
train: 2221
train: 2222
train: 2223
train: 2224
train: 2225
train: 2226
trai

train: 2828
train: 2829
train: 2830
train: 2831
train: 2832
train: 2833
train: 2834
train: 2835
train: 2836
train: 2837
train: 2838
train: 2839
train: 2840
train: 2841
train: 2842
train: 2843
train: 2844
train: 2845
train: 2846
train: 2847
train: 2848
train: 2849
train: 2850
train: 2851
train: 2852
train: 2853
train: 2854
train: 2855
train: 2856
train: 2857
train: 2858
train: 2859
train: 2860
train: 2861
train: 2862
train: 2863
train: 2864
train: 2865
train: 2866
train: 2867
train: 2868
train: 2869
train: 2870
train: 2871
train: 2872
train: 2873
train: 2874
train: 2875
train: 2876
train: 2877
train: 2878
train: 2879
train: 2880
train: 2881
train: 2882
train: 2883
train: 2884
train: 2885
train: 2886
train: 2887
train: 2888
train: 2889
train: 2890
train: 2891
train: 2892
train: 2893
train: 2894
train: 2895
train: 2896
train: 2897
train: 2898
train: 2899
train: 2900
train: 2901
train: 2902
train: 2903
train: 2904
train: 2905
train: 2906
train: 2907
train: 2908
train: 2909
train: 2910
trai

...

train: 6928
train: 6929
train: 6930
train: 6931
train: 6932
train: 6933
train: 6934
train: 6935
train: 6936
train: 6937
train: 6938
train: 6939
train: 6940
train: 6941
train: 6942
train: 6943
train: 6944
train: 6945
train: 6946
train: 6947
train: 6948
train: 6949
train: 6950
train: 6951
train: 6952
train: 6953
train: 6954
train: 6955
train: 6956
train: 6957
train: 6958
train: 6959
train: 6960
train: 6961
train: 6962
train: 6963
train: 6964
train: 6965
train: 6966
train: 6967
train: 6968
train: 6969
train: 6970
train: 6971
train: 6972
train: 6973
train: 6974
train: 6975
train: 6976
train: 6977
train: 6978
train: 6979
train: 6980
train: 6981
train: 6982
train: 6983
train: 6984
train: 6985
train: 6986
train: 6987
train: 6988
train: 6989
train: 6990
train: 6991
train: 6992
train: 6993
train: 6994
train: 6995
train: 6996
train: 6997
train: 6998
train: 6999
train: 7000
train: 7001
train: 7002
train: 7003
train: 7004
train: 7005
train: 7006
train: 7007
train: 7008
train: 7009
train: 7010
trai

train: 7611
train: 7612
train: 7613
train: 7614
train: 7615
train: 7616
train: 7617
train: 7618
train: 7619
train: 7620
train: 7621
train: 7622
train: 7623
train: 7624
train: 7625
train: 7626
train: 7627
train: 7628
train: 7629
train: 7630
train: 7631
train: 7632
train: 7633
train: 7634
train: 7635
train: 7636
train: 7637
train: 7638
train: 7639
train: 7640
train: 7641
train: 7642
train: 7643
train: 7644
train: 7645
train: 7646
train: 7647
train: 7648
train: 7649
train: 7650
train: 7651
train: 7652
train: 7653
train: 7654
train: 7655
train: 7656
train: 7657
train: 7658
train: 7659
train: 7660
train: 7661
train: 7662
train: 7663
train: 7664
train: 7665
train: 7666
train: 7667
train: 7668
train: 7669
train: 7670
train: 7671
train: 7672
train: 7673
train: 7674
train: 7675
train: 7676
train: 7677
train: 7678
train: 7679
train: 7680
train: 7681
train: 7682
train: 7683
train: 7684
train: 7685
train: 7686
train: 7687
train: 7688
train: 7689
train: 7690
train: 7691
train: 7692
train: 7693
trai

train: 8295
train: 8296
train: 8297
train: 8298
train: 8299
train: 8300
train: 8301
train: 8302
train: 8303
train: 8304
train: 8305
train: 8306
train: 8307
train: 8308
train: 8309
train: 8310
train: 8311
train: 8312
train: 8313
train: 8314
train: 8315
train: 8316
train: 8317
train: 8318
train: 8319
train: 8320
train: 8321
train: 8322
train: 8323
train: 8324
train: 8325
train: 8326
train: 8327
train: 8328
train: 8329
train: 8330
train: 8331
train: 8332
train: 8333
train: 8334
train: 8335
train: 8336
train: 8337
train: 8338
train: 8339
train: 8340
train: 8341
train: 8342
train: 8343
train: 8344
train: 8345
train: 8346
train: 8347
train: 8348
train: 8349
train: 8350
train: 8351
train: 8352
train: 8353
train: 8354
train: 8355
train: 8356
train: 8357
train: 8358
train: 8359
train: 8360
train: 8361
train: 8362
train: 8363
train: 8364
train: 8365
train: 8366
train: 8367
train: 8368
train: 8369
train: 8370
train: 8371
train: 8372
train: 8373
train: 8374
train: 8375
train: 8376
train: 8377
trai

train: 8978
train: 8979
train: 8980
train: 8981
train: 8982
train: 8983
train: 8984
train: 8985
train: 8986
train: 8987
train: 8988
train: 8989
train: 8990
train: 8991
train: 8992
train: 8993
train: 8994
train: 8995
train: 8996
train: 8997
train: 8998
train: 8999
train: 9000
train: 9001
train: 9002
train: 9003
train: 9004
train: 9005
train: 9006
train: 9007
train: 9008
train: 9009
train: 9010
train: 9011
train: 9012
train: 9013
train: 9014
train: 9015
train: 9016
train: 9017
train: 9018
train: 9019
train: 9020
train: 9021
train: 9022
train: 9023
train: 9024
train: 9025
train: 9026
train: 9027
train: 9028
train: 9029
train: 9030
train: 9031
train: 9032
train: 9033
train: 9034
train: 9035
train: 9036
train: 9037
train: 9038
train: 9039
train: 9040
train: 9041
train: 9042
train: 9043
train: 9044
train: 9045
train: 9046
train: 9047
train: 9048
train: 9049
train: 9050
train: 9051
train: 9052
train: 9053
train: 9054
train: 9055
train: 9056
train: 9057
train: 9058
train: 9059
train: 9060
trai

train: 9662
train: 9663
train: 9664
train: 9665
train: 9666
train: 9667
train: 9668
train: 9669
train: 9670
train: 9671
train: 9672
train: 9673
train: 9674
train: 9675
train: 9676
train: 9677
train: 9678
train: 9679
train: 9680
train: 9681
train: 9682
train: 9683
train: 9684
train: 9685
train: 9686
train: 9687
train: 9688
train: 9689
train: 9690
train: 9691
train: 9692
train: 9693
train: 9694
train: 9695
train: 9696
train: 9697
train: 9698
train: 9699
train: 9700
train: 9701
train: 9702
train: 9703
train: 9704
train: 9705
train: 9706
train: 9707
train: 9708
train: 9709
train: 9710
train: 9711
train: 9712
train: 9713
train: 9714
train: 9715
train: 9716
train: 9717
train: 9718
train: 9719
train: 9720
train: 9721
train: 9722
train: 9723
train: 9724
train: 9725
train: 9726
train: 9727
train: 9728
train: 9729
train: 9730
train: 9731
train: 9732
train: 9733
train: 9734
train: 9735
train: 9736
train: 9737
train: 9738
train: 9739
train: 9740
train: 9741
train: 9742
train: 9743
train: 9744
trai

In [14]:
#生成特征向量/标签并写入列表中(把图片得每一行/每一列中黑点数作为特征数组成33*80)
train_x = []  #特征向量
train_y = []  #标签
def modetxt(number, num):  #number:数字 num:文件数
    for i in range(num):  #打开每一张图片
        img = Image.open('train_cut_img/'+str(number)+'/'+str(number)+'_ ('+str(i+1)+').jpg', 'r')
        width, height = img.size
        pixel_cnt_list = []  #当前图片的特征向量
        for y in range(height):
            pix_cnt_x = 0
            for x in range(width):
                if img.getpixel((x, y)) == 0:  #黑色像素点
                    pix_cnt_x = pix_cnt_x+1
            pixel_cnt_list.append(pix_cnt_x)
        for x in range(width):
            pix_cnt_y = 0
            for y in range(height):
                if img.getpixel((x, y)) == 0:  # 黑色像素点
                    pix_cnt_y += 1
            pixel_cnt_list.append(pix_cnt_y)
        #特征向量/标签存放进相应的列表中
        train_x.append(pixel_cnt_list)
        train_y.append(number)

In [15]:
#先将train_cut_img 中各文件夹中得各图片文件重命名(0_ (1).jpg, 0_ (2).jpg, 0_(3).jpg...)
#调用函数获取特征向量
modetxt(0, 2799)  #2972
modetxt(1, 2685)  #2975
modetxt(2, 2795)  #3074
modetxt(3, 2811)  #2969
modetxt(4, 2783)  #2952
modetxt(5, 2874)  #2991
modetxt(6, 2889)  #2984
modetxt(7, 2973)  #3075
modetxt(8, 2839)  #2951
modetxt(9, 2917)  #3057
train_x = np.array(train_x)
train_y = np.array(train_y)

In [22]:
data = pd.DataFrame(train_x)
data.to_csv('train_x.csv')
data = pd.DataFrame(train_y)
data.to_csv('train_y.csv')

In [11]:
#读取test图片并处理切割保存
for i in range(0,20000):
    print('test:',i)
    img = Image.open('test/'+str(i)+'.jpg', 'r')
    img = img.resize((99, 80))
    #对图片进行二值化
    table = get_bin_table()
    img = img.convert('L')
    img = img.point(table, '1')
    #进行5次降噪处理 
    for j in range(2):
        noise = collext_noise_point(img)
        remove_noise_pixel(img, noise)
    #分割图片并存储
    width, height = img.size
    pixel_line = []  #图像列中的黑色像素点的个数
    for x in range(width):
        pix_y = 0
        for y in range(height):
            if img.getpixel((x, y)) == 0:  # 黑色像素点
                pix_y += 1
        pixel_line.append(pix_y)
    #找到黑色像素点最少的位置
    a = pixel_line[20]
    x1_pixel = 20
    b = pixel_line[55]
    x2_pixel = 55
    for m in range(20, 44):
        if a>=pixel_line[m]:
            a = pixel_line[m]
            x1_pixel = m
        else:
            continue
    for n in range(55,73):
        if b>=pixel_line[n]:
            b = pixel_line[n]
            x2_pixel = n
        else:
            continue
    #切割将处理后的文件存储在文件夹中
    child_img = img.crop((0, 0, x1_pixel, 80))
    child_img = child_img.resize((33,80))
    child_img.save('test_cut_img_2/'+str(i)+'_0.jpg')
    child_img = img.crop((x1_pixel, 0, x2_pixel, 80))
    child_img = child_img.resize((33,80))
    child_img.save('test_cut_img_2/'+str(i)+'_1.jpg')
    child_img = img.crop((x2_pixel, 0, 99, 80))
    child_img = child_img.resize((33,80)) 
    child_img.save('test_cut_img_2/'+str(i)+'_2.jpg')

test: 0
test: 1
test: 2
test: 3
test: 4
test: 5
test: 6
test: 7
test: 8
test: 9
test: 10
test: 11
test: 12
test: 13
test: 14
test: 15
test: 16
test: 17
test: 18
test: 19
test: 20
test: 21
test: 22
test: 23
test: 24
test: 25
test: 26
test: 27
test: 28
test: 29
test: 30
test: 31
test: 32
test: 33
test: 34
test: 35
test: 36
test: 37
test: 38
test: 39
test: 40
test: 41
test: 42
test: 43
test: 44
test: 45
test: 46
test: 47
test: 48
test: 49
test: 50
test: 51
test: 52
test: 53
test: 54
test: 55
test: 56
test: 57
test: 58
test: 59
test: 60
test: 61
test: 62
test: 63
test: 64
test: 65
test: 66
test: 67
test: 68
test: 69
test: 70
test: 71
test: 72
test: 73
test: 74
test: 75
test: 76
test: 77
test: 78
test: 79
test: 80
test: 81
test: 82
test: 83
test: 84
test: 85
test: 86
test: 87
test: 88
test: 89
test: 90
test: 91
test: 92
test: 93
test: 94
test: 95
test: 96
test: 97
test: 98
test: 99
test: 100
test: 101
test: 102
test: 103
test: 104
test: 105
test: 106
test: 107
test: 108
test: 109
test: 110


test: 831
test: 832
test: 833
test: 834
test: 835
test: 836
test: 837
test: 838
test: 839
test: 840
test: 841
test: 842
test: 843
test: 844
test: 845
test: 846
test: 847
test: 848
test: 849
test: 850
test: 851
test: 852
test: 853
test: 854
test: 855
test: 856
test: 857
test: 858
test: 859
test: 860
test: 861
test: 862
test: 863
test: 864
test: 865
test: 866
test: 867
test: 868
test: 869
test: 870
test: 871
test: 872
test: 873
test: 874
test: 875
test: 876
test: 877
test: 878
test: 879
test: 880
test: 881
test: 882
test: 883
test: 884
test: 885
test: 886
test: 887
test: 888
test: 889
test: 890
test: 891
test: 892
test: 893
test: 894
test: 895
test: 896
test: 897
test: 898
test: 899
test: 900
test: 901
test: 902
test: 903
test: 904
test: 905
test: 906
test: 907
test: 908
test: 909
test: 910
test: 911
test: 912
test: 913
test: 914
test: 915
test: 916
test: 917
test: 918
test: 919
test: 920
test: 921
test: 922
test: 923
test: 924
test: 925
test: 926
test: 927
test: 928
test: 929
test: 930


test: 1593
test: 1594
test: 1595
test: 1596
test: 1597
test: 1598
test: 1599
test: 1600
test: 1601
test: 1602
test: 1603
test: 1604
test: 1605
test: 1606
test: 1607
test: 1608
test: 1609
test: 1610
test: 1611
test: 1612
test: 1613
test: 1614
test: 1615
test: 1616
test: 1617
test: 1618
test: 1619
test: 1620
test: 1621
test: 1622
test: 1623
test: 1624
test: 1625
test: 1626
test: 1627
test: 1628
test: 1629
test: 1630
test: 1631
test: 1632
test: 1633
test: 1634
test: 1635
test: 1636
test: 1637
test: 1638
test: 1639
test: 1640
test: 1641
test: 1642
test: 1643
test: 1644
test: 1645
test: 1646
test: 1647
test: 1648
test: 1649
test: 1650
test: 1651
test: 1652
test: 1653
test: 1654
test: 1655
test: 1656
test: 1657
test: 1658
test: 1659
test: 1660
test: 1661
test: 1662
test: 1663
test: 1664
test: 1665
test: 1666
test: 1667
test: 1668
test: 1669
test: 1670
test: 1671
test: 1672
test: 1673
test: 1674
test: 1675
test: 1676
test: 1677
test: 1678
test: 1679
test: 1680
test: 1681
test: 1682
test: 1683

test: 2340
test: 2341
test: 2342
test: 2343
test: 2344
test: 2345
test: 2346
test: 2347
test: 2348
test: 2349
test: 2350
test: 2351
test: 2352
test: 2353
test: 2354
test: 2355
test: 2356
test: 2357
test: 2358
test: 2359
test: 2360
test: 2361
test: 2362
test: 2363
test: 2364
test: 2365
test: 2366
test: 2367
test: 2368
test: 2369
test: 2370
test: 2371
test: 2372
test: 2373
test: 2374
test: 2375
test: 2376
test: 2377
test: 2378
test: 2379
test: 2380
test: 2381
test: 2382
test: 2383
test: 2384
test: 2385
test: 2386
test: 2387
test: 2388
test: 2389
test: 2390
test: 2391
test: 2392
test: 2393
test: 2394
test: 2395
test: 2396
test: 2397
test: 2398
test: 2399
test: 2400
test: 2401
test: 2402
test: 2403
test: 2404
test: 2405
test: 2406
test: 2407
test: 2408
test: 2409
test: 2410
test: 2411
test: 2412
test: 2413
test: 2414
test: 2415
test: 2416
test: 2417
test: 2418
test: 2419
test: 2420
test: 2421
test: 2422
test: 2423
test: 2424
test: 2425
test: 2426
test: 2427
test: 2428
test: 2429
test: 2430

test: 3086
test: 3087
test: 3088
test: 3089
test: 3090
test: 3091
test: 3092
test: 3093
test: 3094
test: 3095
test: 3096
test: 3097
test: 3098
test: 3099
test: 3100
test: 3101
test: 3102
test: 3103
test: 3104
test: 3105
test: 3106
test: 3107
test: 3108
test: 3109
test: 3110
test: 3111
test: 3112
test: 3113
test: 3114
test: 3115
test: 3116
test: 3117
test: 3118
test: 3119
test: 3120
test: 3121
test: 3122
test: 3123
test: 3124
test: 3125
test: 3126
test: 3127
test: 3128
test: 3129
test: 3130
test: 3131
test: 3132
test: 3133
test: 3134
test: 3135
test: 3136
test: 3137
test: 3138
test: 3139
test: 3140
test: 3141
test: 3142
test: 3143
test: 3144
test: 3145
test: 3146
test: 3147
test: 3148
test: 3149
test: 3150
test: 3151
test: 3152
test: 3153
test: 3154
test: 3155
test: 3156
test: 3157
test: 3158
test: 3159
test: 3160
test: 3161
test: 3162
test: 3163
test: 3164
test: 3165
test: 3166
test: 3167
test: 3168
test: 3169
test: 3170
test: 3171
test: 3172
test: 3173
test: 3174
test: 3175
test: 3176

...

test: 16649
test: 16650
test: 16651
test: 16652
test: 16653
test: 16654
test: 16655
test: 16656
test: 16657
test: 16658
test: 16659
test: 16660
test: 16661
test: 16662
test: 16663
test: 16664
test: 16665
test: 16666
test: 16667
test: 16668
test: 16669
test: 16670
test: 16671
test: 16672
test: 16673
test: 16674
test: 16675
test: 16676
test: 16677
test: 16678
test: 16679
test: 16680
test: 16681
test: 16682
test: 16683
test: 16684
test: 16685
test: 16686
test: 16687
test: 16688
test: 16689
test: 16690
test: 16691
test: 16692
test: 16693
test: 16694
test: 16695
test: 16696
test: 16697
test: 16698
test: 16699
test: 16700
test: 16701
test: 16702
test: 16703
test: 16704
test: 16705
test: 16706
test: 16707
test: 16708
test: 16709
test: 16710
test: 16711
test: 16712
test: 16713
test: 16714
test: 16715
test: 16716
test: 16717
test: 16718
test: 16719
test: 16720
test: 16721
test: 16722
test: 16723
test: 16724
test: 16725
test: 16726
test: 16727
test: 16728
test: 16729
test: 16730
test: 16731
test

test: 17332
test: 17333
test: 17334
test: 17335
test: 17336
test: 17337
test: 17338
test: 17339
test: 17340
test: 17341
test: 17342
test: 17343
test: 17344
test: 17345
test: 17346
test: 17347
test: 17348
test: 17349
test: 17350
test: 17351
test: 17352
test: 17353
test: 17354
test: 17355
test: 17356
test: 17357
test: 17358
test: 17359
test: 17360
test: 17361
test: 17362
test: 17363
test: 17364
test: 17365
test: 17366
test: 17367
test: 17368
test: 17369
test: 17370
test: 17371
test: 17372
test: 17373
test: 17374
test: 17375
test: 17376
test: 17377
test: 17378
test: 17379
test: 17380
test: 17381
test: 17382
test: 17383
test: 17384
test: 17385
test: 17386
test: 17387
test: 17388
test: 17389
test: 17390
test: 17391
test: 17392
test: 17393
test: 17394
test: 17395
test: 17396
test: 17397
test: 17398
test: 17399
test: 17400
test: 17401
test: 17402
test: 17403
test: 17404
test: 17405
test: 17406
test: 17407
test: 17408
test: 17409
test: 17410
test: 17411
test: 17412
test: 17413
test: 17414
test

test: 18015
test: 18016
test: 18017
test: 18018
test: 18019
test: 18020
test: 18021
test: 18022
test: 18023
test: 18024
test: 18025
test: 18026
test: 18027
test: 18028
test: 18029
test: 18030
test: 18031
test: 18032
test: 18033
test: 18034
test: 18035
test: 18036
test: 18037
test: 18038
test: 18039
test: 18040
test: 18041
test: 18042
test: 18043
test: 18044
test: 18045
test: 18046
test: 18047
test: 18048
test: 18049
test: 18050
test: 18051
test: 18052
test: 18053
test: 18054
test: 18055
test: 18056
test: 18057
test: 18058
test: 18059
test: 18060
test: 18061
test: 18062
test: 18063
test: 18064
test: 18065
test: 18066
test: 18067
test: 18068
test: 18069
test: 18070
test: 18071
test: 18072
test: 18073
test: 18074
test: 18075
test: 18076
test: 18077
test: 18078
test: 18079
test: 18080
test: 18081
test: 18082
test: 18083
test: 18084
test: 18085
test: 18086
test: 18087
test: 18088
test: 18089
test: 18090
test: 18091
test: 18092
test: 18093
test: 18094
test: 18095
test: 18096
test: 18097
test

test: 18700
test: 18701
test: 18702
test: 18703
test: 18704
test: 18705
test: 18706
test: 18707
test: 18708
test: 18709
test: 18710
test: 18711
test: 18712
test: 18713
test: 18714
test: 18715
test: 18716
test: 18717
test: 18718
test: 18719
test: 18720
test: 18721
test: 18722
test: 18723
test: 18724
test: 18725
test: 18726
test: 18727
test: 18728
test: 18729
test: 18730
test: 18731
test: 18732
test: 18733
test: 18734
test: 18735
test: 18736
test: 18737
test: 18738
test: 18739
test: 18740
test: 18741
test: 18742
test: 18743
test: 18744
test: 18745
test: 18746
test: 18747
test: 18748
test: 18749
test: 18750
test: 18751
test: 18752
test: 18753
test: 18754
test: 18755
test: 18756
test: 18757
test: 18758
test: 18759
test: 18760
test: 18761
test: 18762
test: 18763
test: 18764
test: 18765
test: 18766
test: 18767
test: 18768
test: 18769
test: 18770
test: 18771
test: 18772
test: 18773
test: 18774
test: 18775
test: 18776
test: 18777
test: 18778
test: 18779
test: 18780
test: 18781
test: 18782
test

test: 19384
test: 19385
test: 19386
test: 19387
test: 19388
test: 19389
test: 19390
test: 19391
test: 19392
test: 19393
test: 19394
test: 19395
test: 19396
test: 19397
test: 19398
test: 19399
test: 19400
test: 19401
test: 19402
test: 19403
test: 19404
test: 19405
test: 19406
test: 19407
test: 19408
test: 19409
test: 19410
test: 19411
test: 19412
test: 19413
test: 19414
test: 19415
test: 19416
test: 19417
test: 19418
test: 19419
test: 19420
test: 19421
test: 19422
test: 19423
test: 19424
test: 19425
test: 19426
test: 19427
test: 19428
test: 19429
test: 19430
test: 19431
test: 19432
test: 19433
test: 19434
test: 19435
test: 19436
test: 19437
test: 19438
test: 19439
test: 19440
test: 19441
test: 19442
test: 19443
test: 19444
test: 19445
test: 19446
test: 19447
test: 19448
test: 19449
test: 19450
test: 19451
test: 19452
test: 19453
test: 19454
test: 19455
test: 19456
test: 19457
test: 19458
test: 19459
test: 19460
test: 19461
test: 19462
test: 19463
test: 19464
test: 19465
test: 19466
test

In [ ]:
#生成特征向量
test_x = []
test_y = []
for i in range(60000):
    img = Image.open('test_cut_img_2/'+'test_ ('+str(i+1)+').jpg', 'r')
    width, height = img.size
    pixel_cnt_list = []  #特征向量
    for y in range(height):
        pix_cnt_x = 0
        for x in range(width):
            if img.getpixel((x, y)) == 0:  #黑色像素点
                pix_cnt_x = pix_cnt_x+1
        pixel_cnt_list.append(pix_cnt_x)
    for x in range(width):
        pix_cnt_y = 0
        for y in range(height):
            if img.getpixel((x, y)) == 0:  # 黑色像素点
                pix_cnt_y += 1
        pixel_cnt_list.append(pix_cnt_y)
    #特征向量/标签存放进相应的列表中
    test_x.append(pixel_cnt_list)

In [85]:
test_x = np.array(test_x)
#调用scikitlearn SVM（正确率较低）
from sklearn import svm
#训练模型
svc = svm.SVC(kernel='linear')
svc.fit(train_x, train_y)
#测试模型得到结果
test_y = svc.predict(test_x)
#保存结果
y = []
i = 0
for j in range(20000):
    a = test_y[i]*100 + test_y[i+1]*10 + test_y[i+2]
    i = i+3
    y.append(a)
y = np.array(y)
data = pd.DataFrame(y)
data.to_csv('y_SVM.csv')

In [ ]:
#Scikitlearn神经网络(正确率16%)
from sklearn import neural_network
mlp_cls = neural_network.MLPClassifier(
    hidden_layer_sizes=[200, 400, 200], max_iter=10000)
mlp_cls.fit(train_x, train_y)
test_y = mlp_cls.predict(test_x)
y = []
i = 0
for j in range(20000):
    a = test_y[i]*100 + test_y[i+1]*10 + test_y[i+2]
    i = i+3
    y.append(a)
y = np.array(y)
data = pd.DataFrame(y)
data.to_csv('y_bp_2.csv')